# Adding Removed S&P 500 Tickers

## Overview:

This step aims to add historically removed S&P500 company tickers to eliminate the survivorship biase among the financial data. 


In [3]:
# import libraries
import yfinance as yf
import pandas as pd
import requests
import json
import numpy as np
import pickle
import copy
import datetime as dt
import os

headers = {"User-Agent": "ian.ye.fu@gmail.com"} 
data_folder_download = './datasets/download/'
data_folder_generate = './datasets/generate/'

## Step 1: Get removed and added tickers

In [4]:
recent_changes_tickers = pd.read_csv(data_folder_download + 'sp500_recent_changes.csv', header = 0, index_col = 'Date')

In [5]:
recent_changes_tickers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 349 entries, Date to June 17, 1997
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Added      336 non-null    object
 1   Added.1    336 non-null    object
 2   Removed    330 non-null    object
 3   Removed.1  330 non-null    object
 4   Reason     349 non-null    object
dtypes: object(5)
memory usage: 16.4+ KB


In [379]:
# Check the overlap between tickers added and tickers removed
set_added_tickers = set(recent_changes_tickers["Added"].tolist())
set_removed_tickers = set(recent_changes_tickers["Removed"].tolist())
print(f'number of ticker overlaps between removed tickers and added tickers: {len(list(set_added_tickers.intersection(set_removed_tickers)))}')

number of ticker overlaps between removed tickers and added tickers: 109


In [6]:
# create added_tickers_df
added_tickers_df = recent_changes_tickers.iloc[1: ][['Added', 'Added.1']].dropna(axis = 0)
added_tickers_df.index = pd.to_datetime(added_tickers_df.index, format='%B %d, %Y')
added_tickers_df = added_tickers_df.reset_index().rename(columns = {'Date':'Date_Added', 'Added': 'Added_Ticker', 'Added.1': 'Security_Added'}).set_index('Added_Ticker')

In [7]:
#Only get the tickers added after 2013 for the analysis period between 2013 to 2024.
added_tickers_df = added_tickers_df.query('Date_Added >= "2013"')
with open(data_folder_generate + 'added_tickers_df.pkl', 'wb') as f: 
    pickle.dump(added_tickers_df, f)

In [8]:
with open(data_folder_generate + 'added_tickers_df.pkl', 'rb') as f: 
    added_tickers_df = pickle.load(f)

In [9]:
added_tickers_df.head()

,Date_Added,Security_Added
Added_Ticker,,
KKR,2024-06-24,KKR
CRWD,2024-06-24,CrowdStrike
GDDY,2024-06-24,GoDaddy
VST,2024-05-08,Vistra
GEV,2024-04-02,GE Vernova


In [10]:
# create removed_tickers_df
removed_tickers_df = recent_changes_tickers.iloc[1: ][['Removed', 'Removed.1']].dropna(axis = 0)
removed_tickers_df.index = pd.to_datetime(removed_tickers_df.index, format='%B %d, %Y')
removed_tickers_df = removed_tickers_df.reset_index().rename(columns = {'Date':'Date_Removed', 'Removed': 'Removed_Ticker', 'Removed.1': 'Security_Removed'}).set_index('Removed_Ticker')

In [11]:
removed_tickers_df = removed_tickers_df.query('Date_Removed >= "2013"')
with open(data_folder_generate + 'removed_tickers_df.pkl', 'wb') as f: 
    pickle.dump(removed_tickers_df, f)

In [12]:
with open(data_folder_generate + 'removed_tickers_df.pkl', 'rb') as f: 
    removed_tickers_df = pickle.load(f)

In [13]:
removed_tickers_df.head()

,Date_Removed,Security_Removed
Removed_Ticker,,
RHI,2024-06-24,Robert Half
CMA,2024-06-24,Comerica
ILMN,2024-06-24,"Illumina, Inc."
PXD,2024-05-08,Pioneer Natural Resources
XRAY,2024-04-03,Dentsply Sirona


## Step 2: Get the CIK numbers for removed tickers

In [14]:
def cik_matching_ticker(ticker, headers=headers):
    """
    get the cik number for tickers
    """
    ticker = ticker.upper().replace(".", "-")
    ticker_json = requests.get(
        "https://www.sec.gov/files/company_tickers.json", headers=headers
    ).json()

    for company in ticker_json.values():
        if company["ticker"] == ticker:
            cik = str(company["cik_str"]).zfill(10)
            return cik
    raise ValueError(f"Ticker {ticker} not found in SEC database")

In [15]:
removed_tickers_list = removed_tickers_df.index.tolist()

In [17]:
for ticker in removed_tickers_list:
    try:
        print(f"Fetching CIK for {ticker}")  
        removed_tickers_df.loc[ticker, 'CIK'] = cik_matching_ticker(ticker, headers)
    except ValueError as e:
        print(f"Error for ticker {ticker}: {e}")
        continue  

Fetching CIK for RHI
Fetching CIK for CMA
Fetching CIK for ILMN
Fetching CIK for PXD
Error for ticker PXD: Ticker PXD not found in SEC database
Fetching CIK for XRAY
Fetching CIK for VFC
Fetching CIK for WHR
Fetching CIK for ZION
Fetching CIK for SEE
Fetching CIK for ALK
Fetching CIK for SEDG
Fetching CIK for OGN
Fetching CIK for ATVI
Error for ticker ATVI: Ticker ATVI not found in SEC database
Fetching CIK for DXC
Fetching CIK for LNC
Fetching CIK for NWL
Fetching CIK for AAP
Fetching CIK for DISH
Error for ticker DISH: Ticker DISH not found in SEC database
Fetching CIK for FRC
Error for ticker FRC: Ticker FRC not found in SEC database
Fetching CIK for LUMN
Fetching CIK for SBNY
Error for ticker SBNY: Ticker SBNY not found in SEC database
Fetching CIK for SIVB
Error for ticker SIVB: Ticker SIVB not found in SEC database
Fetching CIK for VNO
Fetching CIK for ABMD
Error for ticker ABMD: Ticker ABMD not found in SEC database
Fetching CIK for FBHS
Error for ticker FBHS: Ticker FBHS not fo

In [18]:
removed_tickers_df = removed_tickers_df.rename(columns = {'CIK':'removed_cik'})
removed_tickers_df

,Date_Removed,Security_Removed,removed_cik
Removed_Ticker,,,
RHI,2024-06-24,Robert Half,0000315213
CMA,2024-06-24,Comerica,0000028412
ILMN,2024-06-24,"Illumina, Inc.",0001110803
PXD,2024-05-08,Pioneer Natural Resources,nan
XRAY,2024-04-03,Dentsply Sirona,0000818479
...,...,...,...
DF,2013-05-23,Dean Foods,nan
CVH,2013-05-08,Coventry Health Care,nan
PCS,2013-04-30,MetroPCS,nan


In [19]:
with open(data_folder_generate + 'sp500_df.pkl', 'rb') as f: 
    sp500_df = pickle.load(f)

In [20]:
sp500_df.head()

,Security,GICS Sector,GICS Sub-Industry,Date added,CIK,Founded
Symbol,,,,,,
MMM,3M,Industrials,Industrial Conglomerates,1957-03-04,0000066740,1902
AOS,A. O. Smith,Industrials,Building Products,2017-07-26,0000091142,1916
ABT,Abbott Laboratories,Health Care,Health Care Equipment,1957-03-04,0000001800,1888
ABBV,AbbVie,Health Care,Biotechnology,2012-12-31,0001551152,2013 (1888)
ACN,Accenture,Information Technology,IT Consulting & Other Services,2011-07-06,0001467373,1989


## Step 3. Add the removed tickers to the sp500 tickers table

In [25]:
# merge the sp500_df with added_date
merged_df_added_date = pd.merge(left = sp500_df, right = added_tickers_df, left_index = True, right_index = True, how = 'outer')

#merge updated sp500_df with removed_date
merged_df_both_dates = pd.merge(left = merged_df_added_date, right = removed_tickers_df, left_index = True, right_index = True, how = 'outer')

merged_df_both_dates = merged_df_both_dates.reset_index()

merged_df_both_dates.rename(columns = {'Symbol':'Ticker'}, inplace = True)

merged_df_both_dates

,Ticker,Security,GICS Sector,GICS Sub-Industry,Date added,CIK,Founded,Date_Added,Security_Added,Date_Removed,Security_Removed,removed_cik
0,A,Agilent Technologies,Health Care,Life Sciences Tools & Services,2000-06-05,0001090872,1999,NaT,NaN,NaT,NaN,NaN
1,AA,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2016-11-01,Alcoa,0001675149
2,AAL,American Airlines Group,Industrials,Passenger Airlines,2015-03-23,0000006201,1934,2015-03-23,American Airlines Group,NaT,NaN,NaN
3,AAP,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-08,Advance Auto Parts,2023-08-25,Advance Auto Parts,0001158449
4,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals",1982-11-30,0000320193,1977,NaT,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
735,YUM,Yum! Brands,Consumer Discretionary,Restaurants,1997-10-06,0001041061,1997,NaT,NaN,NaT,NaN,NaN
736,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,2001-08-07,0001136869,1927,NaT,NaN,NaT,NaN,NaN
737,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,2019-12-23,0000877212,1969,2019-12-23,Zebra Technologies,NaT,NaN,NaN
738,ZION,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2024-03-18,Zions Bancorporation,0000109380


## Step 4. Clean up the merged sp500 tickers table

In [31]:
# Fill 'Security' with values from 'Security Added' and 'Security Removed'
merged_df_both_dates['Security'] = merged_df_both_dates['Security'].fillna(merged_df_both_dates['Security_Added']).fillna(merged_df_both_dates['Security_Removed'])

# Fill 'sp500cik' with values from 'removed_cik'
merged_df_both_dates['CIK'] = merged_df_both_dates['CIK'].fillna(merged_df_both_dates['removed_cik'])

# Fill 'Date added' with values from 'Date_Added'
merged_df_both_dates['Date added'] = merged_df_both_dates['Date added'].fillna(merged_df_both_dates['Date_Added'])

updated_merged_df_both_dates = merged_df_both_dates[~merged_df_both_dates['CIK'].isna()].copy()

updated_merged_df_both_dates = updated_merged_df_both_dates.drop(columns = ['Date_Added', 'Security_Added', 'Security_Removed', 'removed_cik'])

# filter out the tickers that don't have cik code on SEC.
updated_merged_df_both_dates_df = updated_merged_df_both_dates.loc[updated_merged_df_both_dates['CIK'] != "nan"].copy()

# Add the GICS sector label to the newly added tickers
updated_merged_df_both_dates_df['GICS Sector'].fillna(updated_merged_df_both_dates_df['Ticker'].map(removed_tickers_GICS_sector_category), inplace=True)

In [36]:
updated_merged_df_both_dates_df = updated_merged_df_both_dates_df.rename(columns = {'Date added':'Date_Added'})
new_column_order = ['Ticker', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Date_Added', 'Date_Removed', 'Founded', 'CIK']
reindexed_merged_sp500_df = updated_merged_df_both_dates_df[new_column_order].set_index('Ticker')
reindexed_merged_sp500_df

,Security,GICS Sector,GICS Sub-Industry,Date_Added,Date_Removed,Founded,CIK
Ticker,,,,,,,
A,Agilent Technologies,Health Care,Life Sciences Tools & Services,2000-06-05,NaT,1999,0001090872
AA,Alcoa,Materials,NaN,NaT,2016-11-01,NaN,0001675149
AAL,American Airlines Group,Industrials,Passenger Airlines,2015-03-23,NaT,1934,0000006201
AAP,Advance Auto Parts,Consumer Discretionary,NaN,2015-07-08 00:00:00,2023-08-25,NaN,0001158449
AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals",1982-11-30,NaT,1977,0000320193
...,...,...,...,...,...,...,...
YUM,Yum! Brands,Consumer Discretionary,Restaurants,1997-10-06,NaT,1997,0001041061
ZBH,Zimmer Biomet,Health Care,Health Care Equipment,2001-08-07,NaT,1927,0001136869
ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,2019-12-23,NaT,1969,0000877212


In [48]:
reindexed_merged_sp500_df = reindexed_merged_sp500_df.query('Security != "Under Armour"')

In [50]:
with open(data_folder_generate + 'sp500_df_v2.0.pkl', 'wb') as f:
    pickle.dump(reindexed_merged_sp500_df, f)

In [33]:
removed_tickers_GICS_sector_category = {
    'AA': 'Materials',
    'AAP': 'Consumer Discretionary',
    'ABMD': 'Health Care',
    'ACE': 'Financials',
    'ADS': 'Information Technology',
    'ADT': 'Industrials',
    'AET': 'Health Care',
    'AGN': 'Health Care',
    'AIV': 'Real Estate',
    'ALK': 'Industrials',
    'ALTR': 'Information Technology',
    'ALXN': 'Health Care',
    'AMG': 'Financials',
    'AN': 'Consumer Discretionary',
    'ANDV': 'Energy',
    'ANF': 'Consumer Discretionary',
    'APC': 'Energy',
    'APOL': 'Consumer Discretionary',
    'ARG': 'Materials',
    'ARNC': 'Materials',
    'ATI': 'Materials',
    'ATVI': 'Communication Services',
    'AVP': 'Consumer Staples',
    'AYI': 'Industrials',
    'BBBY': 'Consumer Discretionary',
    'BCR': 'Health Care',
    'BEAM': 'Health Care',
    'BHF': 'Financials',
    'BIG': 'Consumer Discretionary',
    'BMC': 'Information Technology',
    'BMS': 'Materials',
    'BRCM': 'Information Technology',
    'BTU': 'Energy',
    'BXLT': 'Health Care',
    'CA': 'Information Technology',
    'CAM': 'Energy',
    'CCE': 'Consumer Staples',
    'CDAY': 'Information Technology',
    'CELG': 'Health Care',
    'CERN': 'Health Care',
    'CFN': 'Health Care',
    'CHK': 'Energy',
    'CLF': 'Materials',
    'CMA': 'Financials',
    'CMCSK': 'Communication Services',
    'CNX': 'Energy',
    'COL': 'Industrials',
    'COTY': 'Consumer Staples',
    'COV': 'Health Care',
    'CPGX': 'Energy',
    'CPRI': 'Consumer Discretionary',
    'CSC': 'Information Technology',
    'CSRA': 'Information Technology',
    'CTXS': 'Information Technology',
    'CVC': 'Communication Services',
    'CVH': 'Health Care',
    'CXO': 'Energy',
    'DELL': 'Information Technology',
    'DF': 'Consumer Staples',
    'DISCK': 'Communication Services',
    'DISH': 'Communication Services',
    'DNB': 'Information Technology',
    'DNR': 'Energy',
    'DO': 'Energy',
    'DPS': 'Consumer Staples',
    'DRE': 'Real Estate',
    'DTV': 'Communication Services',
    'DWDP': 'Materials',
    'DXC': 'Information Technology',
    'EMC': 'Information Technology',
    'ENDP': 'Health Care',
    'ESRX': 'Health Care',
    'ESV': 'Energy',
    'ETFC': 'Financials',
    'EVHC': 'Health Care',
    'FB': 'Communication Services',
    'FBHS': 'Consumer Discretionary',
    'FDO': 'Consumer Discretionary',
    'FHN': 'Financials',
    'FII': 'Financials',
    'FL': 'Consumer Discretionary',
    'FLIR': 'Information Technology',
    'FLR': 'Industrials',
    'FLS': 'Industrials',
    'FLT': 'Information Technology',
    'FOSL': 'Consumer Discretionary',
    'FRC': 'Financials',
    'FRX': 'Health Care',
    'FTI': 'Energy',
    'FTR': 'Communication Services',
    'GAS': 'Utilities',
    'GGP': 'Real Estate',
    'GHC': 'Consumer Discretionary',
    'GMCR': 'Consumer Staples',
    'GME': 'Consumer Discretionary',
    'GNW': 'Financials',
    'GPS': 'Consumer Discretionary',
    'GT': 'Consumer Discretionary',
    'HAR': 'Consumer Discretionary',
    'HBI': 'Consumer Discretionary',
    'HCBK': 'Financials',
    'HFC': 'Energy',
    'HNZ': 'Consumer Staples',
    'HOG': 'Consumer Discretionary',
    'HOT': 'Consumer Discretionary',
    'HP': 'Energy',
    'HRB': 'Financials',
    'HSP': 'Health Care',
    'IGT': 'Consumer Discretionary',
    'ILMN': 'Health Care',
    'INFO': 'Information Technology',
    'IPGP': 'Information Technology',
    'JCP': 'Consumer Discretionary',
    'JDSU': 'Information Technology',
    'JEF': 'Financials',
    'JOY': 'Industrials',
    'JWN': 'Consumer Discretionary',
    'KORS': 'Consumer Discretionary',
    'KRFT': 'Consumer Staples',
    'KSS': 'Consumer Discretionary',
    'KSU': 'Industrials',
    'LEG': 'Consumer Discretionary',
    'LIFE': 'Health Care',
    'LLL': 'Industrials',
    'LLTC': 'Information Technology',
    'LM': 'Financials',
    'LNC': 'Financials',
    'LO': 'Consumer Staples',
    'LSI': 'Real Estate',
    'LUMN': 'Information Technology',
    'LVLT': 'Information Technology',
    'M': 'Consumer Discretionary',
    'MAC': 'Real Estate',
    'MAT': 'Consumer Discretionary',
    'MBC': 'Health Care',
    'MJN': 'Consumer Staples',
    'MNK': 'Health Care',
    'MOLX': 'Information Technology',
    'MON': 'Materials',
    'MUR': 'Energy',
    'MXIM': 'Information Technology',
    'NAVI': 'Financials',
    'NBL': 'Energy',
    'NBR': 'Energy',
    'NE': 'Energy',
    'NFX': 'Energy',
    'NKTR': 'Health Care',
    'NLSN': 'Communication Services',
    'NOV': 'Energy',
    'NWL': 'Consumer Discretionary',
    'NYX': 'Financials',
    'OGN': 'Health Care',
    'OI': 'Materials',
    'PBCT': 'Financials',
    'PBI': 'Industrials',
    'PCL': 'Real Estate',
    'PCP': 'Industrials',
    'PCS': 'Communication Services',
    'PDCO': 'Health Care',
    'PENN': 'Consumer Discretionary',
    'PETM': 'Consumer Discretionary',
    'PLL': 'Materials',
    'POM': 'Utilities',
    'PRGO': 'Health Care',
    'PVH': 'Consumer Discretionary',
    'PXD': 'Energy',
    'Q': 'Communication Services',
    'QEP': 'Energy',
    'R': 'Industrials',
    'RAI': 'Consumer Staples',
    'RDC': 'Energy',
    'RE': 'Financials',
    'RHI': 'Industrials',
    'RHT': 'Information Technology',
    'RIG': 'Energy',
    'RRC': 'Energy',
    'RTN': 'Industrials',
    'S': 'Communication Services',
    'SAI': 'Information Technology',
    'SBNY': 'Financials',
    'SCG': 'Utilities',
    'SE': 'Communication Services',
    'SEDG': 'Information Technology',
    'SEE': 'Materials',
    'SIAL': 'Materials',
    'SIG': 'Consumer Discretionary',
    'SIVB': 'Financials',
    'SLG': 'Real Estate',
    'SLM': 'Financials',
    'SNDK': 'Information Technology',
    'SNI': 'Communication Services',
    'SPLS': 'Consumer Discretionary',
    'SRCL': 'Industrials',
    'STI': 'Financials',
    'STJ': 'Health Care',
    'SWN': 'Energy',
    'SWY': 'Consumer Staples',
    'TDC': 'Information Technology',
    'TE': 'Utilities',
    'TEG': 'Utilities',
    'TGNA': 'Communication Services',
    'THC': 'Health Care',
    'TIF': 'Consumer Discretionary',
    'TRIP': 'Communication Services',
    'TSS': 'Information Technology',
    'TWC': 'Communication Services',
    'TWTR': 'Communication Services',
    'TWX': 'Communication Services',
    'TYC': 'Industrials',
    'UA': 'Consumer Discretionary',
    'UAA': 'Consumer Discretionary',
    'UNM': 'Financials',
    'URBN': 'Consumer Discretionary',
    'VAR': 'Health Care',
    'VFC': 'Consumer Discretionary',
    'VIAB': 'Communication Services',
    'VNO': 'Real Estate',
    'VNT': 'Industrials',
    'WCG': 'Health Care',
    'WFM': 'Consumer Staples',
    'WHR': 'Consumer Discretionary',
    'WIN': 'Communication Services',
    'WLTW': 'Financials',
    'WPX': 'Energy',
    'WU': 'Information Technology',
    'WYN': 'Consumer Discretionary',
    'X': 'Materials',
    'XEC': 'Energy',
    'XL': 'Financials',
    'XLNX': 'Information Technology',
    'XRAY': 'Health Care',
    'XRX': 'Information Technology',
    'YHOO': 'Communication Services',
    'ZION': 'Financials'
}